In [ ]:
!pip install -q joblib=='1.2.0'
!pip install catboost=='1.1'

In [1]:
import pandas as pd
from sklearn import preprocessing
import pandas as pd
from sklearn.metrics import f1_score
import joblib
from sklearn.preprocessing import OrdinalEncoder
import os
import catboost
import numpy as np
import seaborn as sns
from catboost import CatBoostClassifier, Pool, sum_models
from catboost import Pool, cv
print(joblib.__version__)
print(catboost.__version__)

1.2.0
1.1


In [2]:
root='/kaggle/input/ds-task-1/'
path_train_norm=os.path.join(root,'train_dataset_train_norm.csv')
path_test_norm=os.path.join(root,'test_dataset_test_norm.csv')
path_subm=os.path.join(root,'sample_solution.csv')
path_le=os.path.join(root,'labelEncoder.joblib')
enc=joblib.load(path_le)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [6]:
df_train=pd.read_csv(path_train_norm)
df_test=pd.read_csv(path_test_norm)
# df_train=pd.read_csv(path_train).drop(['id'],axis=1)
# df_test=pd.read_csv(path_test).drop(['id'],axis=1)
df_subm=pd.read_csv(path_subm,index_col='id')
df_train.head()

,Easting,Northing,Height,Reflectance,Class
0,-0.343968,0.394057,-0.078925,-0.134763,0.0
1,0.121019,-0.572918,-0.189725,-1.024600,2.0
2,-0.334820,0.494377,-0.068924,-0.677077,0.0
3,-0.884090,1.170810,-0.093640,-0.150258,0.0
4,0.447037,-0.662854,-0.914836,0.316796,0.0


In [7]:
y=df_train[['Class']]
df_train=df_train.drop(['Class'],axis=1)

In [8]:
for df in [df_train,df_test]:
    df['log_Reflectance_10']=np.log(df.Reflectance+10)  
    #####################
    denominator=np.sqrt(np.power(df.Easting,2)+np.power(df.Northing,2)+np.power(df.Height,2))
    df['Phi']=(df.Easting+df.Northing+df.Height)/denominator  
    df['ENH']=df.Easting*df.Northing/df.Height  
    ####################################################
    df['cos_E']=np.cos(df.Easting)
    df['cos_N']=np.cos(df.Northing)
    df['cos_H']=np.cos(df.Height)
    #####################################################
    df['sin_E']=np.sin(df.Easting)
    df['sin_N']=np.sin(df.Northing)
    df['sin_H']=np.sin(df.Height)
    #####################################################
    #denominator=np.sqrt(np.power(df.Easting,2)+np.power(df.Height,2))
    #df['Phi_EH']=(df.Easting+df.Height)/denominator  
    #denominator=np.sqrt(np.power(df.Northing,2)+np.power(df.Height,2))
    #df['Phi_NH']=(df.Northing+df.Height)/denominator  
    denominator=np.sqrt(np.power(df.Easting,2)+np.power(df.Northing,2))
    df['Phi_EN']=(df.Easting+df.Northing)/denominator  

    ###################
    denominator=np.sqrt(np.power(df.Easting,2)+np.power(df.Northing,2)+np.power(df.Height,2))
    df['Phi_abs']=(abs(df.Easting)+abs(df.Northing)+abs(df.Height))/denominator  
    denominator=np.sqrt(np.power(df.Easting,2)+np.power(df.Northing,2))
    df['Phi_EN_abs']=(abs(df.Easting)+abs(df.Northing))/denominator  


    #####################
    denominator=np.sqrt(np.sqrt(abs(df.Easting))+np.sqrt(abs(df.Northing))+np.sqrt(abs(df.Height)))
    df['Phi_sqrt']=(df.Easting+df.Northing+df.Height)/denominator  
    df['Phi_ccs_sqrt']=(df.cos_E+df.cos_N+df.sin_H)/denominator  #one sine?
    df['Phi_csc_sqrt']=(df.cos_E+df.sin_N+df.cos_H)/denominator  
    df['Phi_scc_sqrt']=(df.sin_E+df.cos_N+df.cos_H)/denominator  

    ########################

    df.drop(['Reflectance','Easting','Northing','sin_H'],axis=1,inplace=True)

In [9]:
df_train.head()

,Height,log_Reflectance_10,Phi,ENH,cos_E,cos_N,cos_H,sin_E,sin_N,Phi_EN,Phi_abs,Phi_EN_abs,Phi_sqrt,Phi_ccs_sqrt,Phi_csc_sqrt,Phi_scc_sqrt
0,-0.078925,2.289017,-0.054513,1.717364,0.941424,0.923359,0.996887,-0.337226,0.383938,0.095760,1.544376,1.410968,-0.023583,1.460570,1.899172,1.294620
1,-0.189725,2.194487,-1.042394,0.365445,0.992686,0.840323,0.982056,0.120724,-0.542086,-0.771738,1.435613,1.185082,-0.516974,1.324954,1.154330,1.565611
2,-0.068924,2.232476,0.150789,2.401577,0.944469,0.880265,0.997626,-0.328599,0.474483,0.267225,1.494249,1.388737,0.072932,1.412949,1.944627,1.246719
3,-0.093640,2.287445,0.131338,11.054095,0.633993,0.389406,0.995619,-0.773338,0.921066,0.195432,1.461497,1.400645,0.126537,0.609417,1.671613,0.400875
4,-0.914836,2.333773,-0.930604,0.323906,0.901732,0.788239,0.609921,0.432296,-0.615369,-0.269936,1.666489,1.388213,-0.723939,0.574661,0.573877,1.172012


In [10]:
import catboost 
version=''
params={
    'auto_class_weights': 'Balanced',
    #'loss_function':'MultiClass',
   # 'boosting_type': 'Ordered',
   # 'thread_count': -1,   
    #'loss_function': 'MultiClassOneVsAll',
    'random_seed':42,
    'task_type':'GPU',
    'eval_metric': 'TotalF1:average=Macro',#'MCC',#PRAUC:type=OneVsAll;use_weights=True
    #'cat_features':list(df_train_cat),    
    'bootstrap_type':'Bernoulli',#'Poisson',
    #'subsample':0.44,  #  ?????????? TRY TO TUNE THIS
    'l2_leaf_reg': 3,
    'early_stopping_rounds':100,
    'iterations':8000,         
     'verbose':300,
     'learning_rate':0.04,
     'depth':7,
     #'save_snapshot':True,
     #'snapshot_file':os.path.join(path_chkpts,f'shapshot_more_rem{version}.bkp'),
     #'snapshot_interval':100,
     'border_count':254,
     #'per_float_feature_quantization':'2:border_count=1024'
    
}
model_catboost=catboost.CatBoostClassifier(**params)#eval_metric='AUC',

In [11]:
grid_search_catbost=model_catboost.fit(X=df_train,y=y)

0:	learn: 0.8230633	total: 50.3ms	remaining: 6m 42s
300:	learn: 0.9527206	total: 13.9s	remaining: 5m 55s
600:	learn: 0.9693385	total: 28.3s	remaining: 5m 48s
900:	learn: 0.9769924	total: 42s	remaining: 5m 31s
1200:	learn: 0.9804124	total: 56.8s	remaining: 5m 21s
1500:	learn: 0.9826182	total: 1m 10s	remaining: 5m 5s
1800:	learn: 0.9839833	total: 1m 24s	remaining: 4m 52s
2100:	learn: 0.9848610	total: 1m 38s	remaining: 4m 36s
2400:	learn: 0.9855574	total: 1m 52s	remaining: 4m 21s
2700:	learn: 0.9861282	total: 2m 6s	remaining: 4m 8s
3000:	learn: 0.9866278	total: 2m 20s	remaining: 3m 53s
3300:	learn: 0.9870322	total: 2m 34s	remaining: 3m 40s
3600:	learn: 0.9873771	total: 2m 48s	remaining: 3m 25s
3900:	learn: 0.9877099	total: 3m 2s	remaining: 3m 11s
4200:	learn: 0.9879993	total: 3m 15s	remaining: 2m 57s
4500:	learn: 0.9882938	total: 3m 29s	remaining: 2m 42s
4800:	learn: 0.9885517	total: 3m 43s	remaining: 2m 29s
5100:	learn: 0.9888112	total: 3m 57s	remaining: 2m 14s
5400:	learn: 0.9890196	tot

In [12]:
f_impt=dict()
k='exp1'
f_impt[k]={'feature_names': df_train.columns,'feature_importance': model_catboost.get_feature_importance()}#).sort_values(['feature_importance'])
pd.DataFrame(f_impt[k]).sort_values(['feature_importance'])

,feature_names,feature_importance
2,Phi,4.444565
8,sin_N,4.707707
3,ENH,4.768074
7,sin_E,4.820610
4,cos_E,4.964607
5,cos_N,5.345657
10,Phi_abs,5.461880
15,Phi_scc_sqrt,5.940952
14,Phi_csc_sqrt,6.067283
0,Height,6.187581


In [13]:
dasdas

NameError: name 'dasdas' is not defined

In [14]:
y_pred=np.argmax(model_catboost.predict_proba(df_test),axis=1)
y_pred=y_pred.reshape(len(y_pred),-1)

In [15]:
df_subm.Class= enc.inverse_transform(y_pred)

In [16]:
version='55'#9k'
df_subm.to_csv(f'my_submission_{version}.csv')
model_catboost.save_model(f'catboost_{version}.ckpt')